---
# First Try

In [ ]:
import numpy as np
import itertools
import random

In [ ]:
# Define the PuzzleBox class
class PuzzleBox:
    def __init__(self, size=(5, 5, 5)):
        self.grid = np.zeros(size, dtype=int)
        self.size = size

    def can_place(self, cube, position):
        """Check if a cube can be placed at the given position."""
        x, y, z = position
        shape = cube.shape  # This now correctly gets the (x, y, z) dimensions as integers
        #print(f"Shape of cube being placed: {shape}")

        # Check if the cube fits within the box by comparing dimensions correctly
        if (x + shape[0] > self.size[0] or
            y + shape[1] > self.size[1] or
            z + shape[2] > self.size[2]):
            return False

        # Check for overlap
        sub_grid = self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]]
        return np.all(sub_grid == 0)

    def place_cube(self, cube, position):
        """Place the cube on the grid."""
        x, y, z = position
        shape = cube.shape  # Get the shape (dimensions)
        self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]] = 1

    def remove_cube(self, cube, position):
        """Remove the cube from the grid."""
        x, y, z = position
        shape = cube.shape
        self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]] = 0

In [ ]:
# Define the Cube class
class Cube:
    def __init__(self, shape):
        self.shape = np.array(shape).shape  # Store the dimensions (not the full array)

    def get_rotations(self):
        """Generate all possible rotations of the cube."""
        rotations = []
        # Iterate over all permutations of axes and flips for 3D rotations
        for axes in itertools.permutations([0, 1, 2]):
            for flip in range(2):
                rotated_shape = np.transpose(np.ones(self.shape), axes)
                if flip:
                    rotated_shape = np.flip(rotated_shape, axis=0)
                rotations.append(rotated_shape.shape)  # Return the shape dimensions
        return rotations

In [ ]:
# Brute force placement function
def solve_puzzle(box, cube_types, remaining_cubes, depth=0):
    if not remaining_cubes:
        # If no remaining cubes, puzzle is solved
        return True

    cube = cube_types[remaining_cubes[0]]
    rotations = cube.get_rotations()

    # Try placing the cube in each position and each rotation
    for rotation in rotations:
        cube.shape = rotation
        for x in range(box.size[0]):
            for y in range(box.size[1]):
                for z in range(box.size[2]):
                    if box.can_place(cube, (x, y, z)):
                        box.place_cube(cube, (x, y, z))
                        if solve_puzzle(box, cube_types, remaining_cubes[1:], depth + 1):
                            return True
                        box.remove_cube(cube, (x, y, z))

    return False

In [ ]:
# Define cube types with explicitly shaped arrays
cubes = [
    Cube(np.ones((3, 2, 2))),  # 4x3x3 cube
    Cube(np.ones((4, 2, 1))),  # 4x2x1 cube
    #Cube(np.ones((1, 1, 1)))   # 1x1x1 cube
]

# Initialize the puzzle box
box = PuzzleBox(size=(5, 5, 5))

# Define the cubes (with the right count)
#cube_types = [0, 1, 2]
cube_types = [0, 1]
remaining_cubes = [0]*6 + [1]*6 #+ [2]*5

# Solve the puzzle
if solve_puzzle(box, cubes, remaining_cubes):
    print("Solution found!")
    print(box.grid)
else:
    print("No solution found.")

No solution found.


---
# Second Try

In [ ]:
import numpy as np
import itertools
import copy  # Used to create copies of the box
import random

In [ ]:
# Cube dictionary (key is cube index, value is the cube's shape)
cube_dict = {
    1: (3, 2, 2),
    2: (3, 2, 2),
    3: (3, 2, 2),
    4: (3, 2, 2),
    5: (3, 2, 2),
    6: (3, 2, 2),
    7: (4, 2, 1),
    8: (4, 2, 1),
    9: (4, 2, 1),
    10: (4, 2, 1),
    11: (4, 2, 1),
    12: (4, 2, 1),
    13: (1, 1, 1),
    14: (1, 1, 1),
    15: (1, 1, 1),
    16: (1, 1, 1),
    17: (1, 1, 1),
}

cube_keys = list(cube_dict.keys())

In [ ]:
def cube_rotation(cube_dimensions):
    """Generate all unique rotations (permutations) for a given cube based on its dimensions."""
    # Use a set to avoid duplicate rotations
    return list(set(itertools.permutations(cube_dimensions, 3)))

In [ ]:
# Define the PuzzleBox class
class PuzzleBox:
    def __init__(self, size=(5, 5, 5)):
        self.grid = np.zeros(size, dtype=int)  # Initialize the 3D grid
        self.size = size

    def possible_positions(self, shape):
        """Check for all possible positions where the cube of the given shape can fit."""
        positions = []
        x_max, y_max, z_max = self.size[0] - shape[0], self.size[1] - shape[1], self.size[2] - shape[2]

        # Iterate over all possible positions where the cube can be placed
        for x in range(x_max + 1):
            for y in range(y_max + 1):
                for z in range(z_max + 1):
                    # Get the sub-grid slice
                    sub_grid = self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]]
                    # Check if all elements in the sub-grid are 0 (empty)
                    if np.all(sub_grid == 0):
                        positions.append((x, y, z))

        return positions

    def place_cube_at_position(self, cube_key, shape, position):
        """Place the cube from the dictionary at the given position in the grid."""
        x, y, z = position
        # Set the grid values to the cube's key
        self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]] = cube_key

    def is_filled(self):
        """Check if the box is fully filled."""
        return np.all(self.grid > 0)

In [ ]:
# Recursive function to solve the puzzle
def solve_puzzle(box, cube_dict, cube_keys, depth=0):
    # Base case: if all cubes are placed, we are done
    if depth == len(cube_keys):
        print("Puzzle solved!")
        print(box.grid)
        return True

    cube_key = cube_keys[depth]
    cube_shape = cube_dict[cube_key]

    # Get all rotations of the current cube
    rotations = cube_rotation(cube_shape)

    # Try placing the cube at all possible positions for each rotation
    for rotation in rotations:
        positions = box.possible_positions(rotation)

        for position in positions:
            # Make a deep copy of the box to keep the current state intact
            new_box = copy.deepcopy(box)

            # Place the cube at the current position in the new box
            new_box.place_cube_at_position(cube_key, rotation, position)

            # Recursively solve the puzzle with the next cube
            if solve_puzzle(new_box, cube_dict, cube_keys, depth + 1):
                return True

    # If no solution found at this level, return False
    return False

In [ ]:
puzzle = PuzzleBox()

if not solve_puzzle(puzzle, cube_dict, cube_keys):
    print("No solution found.")

In [ ]:
def try_random_iteration(box, cube_dict, cube_keys):
    """Try to fill the box by placing cubes in random positions and rotations without recursion."""

    for cube_key in cube_keys:
        cube_shape = cube_dict[cube_key]

        # Get all rotations of the current cube
        rotations = cube_rotation(cube_shape)

        # Shuffle rotations to try them in random order
        random.shuffle(rotations)

        placed = False

        # Try each rotation in a random order
        for rotation in rotations:
            positions = box.possible_positions(rotation)

            # If there are no available positions, break
            if not positions:
                #print(f"No valid position for cube {cube_key}. Aborting attempt.")
                return False

            # Pick a random available position
            position = random.choice(positions)

            # Place the cube at the chosen random position
            box.place_cube_at_position(cube_key, rotation, position)
            placed = True
            break  # Break out after placing this cube

        # If the cube couldn't be placed, stop the process
        if not placed:
            #print(f"Failed to place cube {cube_key}. Aborting attempt.")
            return False

    # If all cubes are placed and the box is filled
    if box.is_filled():
        #print("Box successfully filled!")
        print(box.grid)
        return True

    #print("Box not fully filled but all cubes placed.")
    return False

In [ ]:
puzzle = PuzzleBox()

# Try the random iteration of filling the box
try_random_iteration(puzzle, cube_dict, cube_keys)

False

In [ ]:
solution = False
while not solution:
  puzzle = PuzzleBox()
  solution = try_random_iteration(puzzle, cube_dict, cube_keys)

KeyboardInterrupt: 

It is horrible slow!

# Third Try

In [ ]:
import numpy as np
import itertools
import copy  # Used to create copies of the box
import random

In [ ]:
cubes = {
    (3, 2, 2): 6,
    (4, 2, 1): 6,
    (1, 1, 1): 5,
}
cubes_keys = list(cubes.keys())
print(cubes_keys[0])
print(cubes[cubes_keys[0]])

(3, 2, 2)
6


In [ ]:
def cube_rotation(cube_dimensions):
    """Generate all unique rotations (permutations) for a given cube based on its dimensions."""
    # Use a set to avoid duplicate rotations
    return list(set(itertools.permutations(cube_dimensions, 3)))

In [ ]:
# Define the PuzzleBox class
class PuzzleBox:
    def __init__(self, size=(5, 5, 5)):
        self.grid = np.zeros(size, dtype=int)  # Initialize the 3D grid
        self.size = size

    def get_grid(self):
      return self.grid

    def set_grid(self, grid):
      self.grid = grid

    def possible_positions(self, shape):
        """Check for all possible positions where the cube of the given shape can fit."""
        positions = []
        x_max, y_max, z_max = self.size[0] - shape[0], self.size[1] - shape[1], self.size[2] - shape[2]

        # Iterate over all possible positions where the cube can be placed
        for x in range(x_max + 1):
            for y in range(y_max + 1):
                for z in range(z_max + 1):
                    # Get the sub-grid slice
                    sub_grid = self.grid[x:x + shape[0], y:y + shape[1], z:z + shape[2]]
                    # Check if all elements in the sub-grid are 0 (empty)
                    if np.all(sub_grid == 0):
                        positions.append((x, y, z))

        return positions

    def place_cube_at_position(self, cube_key, shape, position):
        """Place the cube with key at the given position in the grid, trying all rotations in random order."""

        # Get all possible rotations for the given cube shape
        rotations = cube_rotation(shape)

        # Shuffle the rotations to try them in random order
        random.shuffle(rotations)

        # Try each rotation and place the cube in the first available fit
        for rotation in rotations:
            x, y, z = position

            # Check if the cube can fit at the given position with the current rotation
            if (x + rotation[0] <= self.size[0] and
                y + rotation[1] <= self.size[1] and
                z + rotation[2] <= self.size[2]):

                # Check if the space is free (all values are 0 in the relevant sub-grid)
                sub_grid = self.grid[x:x + rotation[0], y:y + rotation[1], z:z + rotation[2]]
                if np.all(sub_grid == 0):
                    # Place the cube at the position with this rotation
                    self.grid[x:x + rotation[0], y:y + rotation[1], z:z + rotation[2]] = cube_key
                    return True  # Successfully placed

        # If no rotation fits, return False (optional, depending on how you want to handle it)
        return False


    def is_filled(self):
        """Check if the box is fully filled."""
        return np.all(self.grid > 0)

In [ ]:
# Function to fit cubes into the box with position checking
def fit_cubes_in_box(box, shape, number, value):
    """Try to place all cubes into the box and return the grid if successful."""

    for remaining_cubes in range(number, 0, -1):
        rotations = cube_rotation(shape)

        # Check if enough space is available
        positions=[]
        for rotation in rotations:
          positions += box.possible_positions(rotation)
        if len(positions) < remaining_cubes:
          return False

        if box.place_cube_at_position(value, rotation, random.choice(positions)):
            continue
        print("error")
        return False

    # If all cubes were placed, return the final grid configuration
    #print(box.get_grid())
    return box.get_grid()

In [ ]:
# List to store all unique configurations
configurations = []

# Run the fit_cubes_in_box function N times
for i in range(3000):
    # Reset the puzzle box grid before each attempt
    puzzle = PuzzleBox()

    # Try to fit the cubes in the box
    result = fit_cubes_in_box(puzzle, cubes_keys[0], cubes[cubes_keys[0]], 1)

    # If a valid configuration is returned, store it in the list
    if result is not False:
        configurations.append(result)

# Print the number of unique configurations found
print(f"Total configurations found: {len(configurations)}")

Total configurations found: 2479


In [ ]:
len(set(tuple(i.flatten()) for i in configurations))

2472

In [ ]:
# Set to store unique configurations in flattened tuple form
unique_configurations_set = set()
# List to store unique configurations as numpy arrays
unique_configurations = []

for config in configurations:
    # Convert the numpy array to a flattened tuple
    config_tuple = tuple(config.flatten())

    # Check if this configuration is unique
    if config_tuple not in unique_configurations_set:
        unique_configurations_set.add(config_tuple)  # Add the unique flattened tuple to the set
        unique_configurations.append(config)         # Store the original numpy array

# Print the number of unique configurations
print(f"Total unique configurations: {len(unique_configurations)}")

Total unique configurations: 2472


In [ ]:
def can_still_fit(box, shape, number):
    """Check if at least 6 long cubes can fit into the given box configuration."""

    rotations = cube_rotation(shape)
    # Check if enough space is available
    positions=[]
    for rotation in rotations:
        positions += box.possible_positions(rotation)
    if len(positions) < number:
        return False

    return True

In [ ]:
# Filter the configurations list
valid_configurations = []
for config in configurations:
    # Reset the puzzle box with the current configuration
    puzzle.set_grid(np.array(config))

    # Check if the current configuration can fit 6 long cubes
    if can_still_fit(puzzle, cubes_keys[1], cubes[cubes_keys[1]]):
        valid_configurations.append(config)

# Print the number of valid configurations
print(f"Valid configurations that can fit 6 long cubes: {len(valid_configurations)}")

Valid configurations that can fit 6 long cubes: 2270


In [ ]:
last_configurations = []
for config in configurations:
    for i in range(50):
        # Reset the puzzle box with the current configuration
        puzzle.set_grid(np.array(config))
        # Try to fit the cubes in the box
        result = fit_cubes_in_box(puzzle, cubes_keys[1], cubes[cubes_keys[1]], 2)

        # If a valid configuration is returned, store it in the list
        if result is not False:
            last_configurations.append(result)
            print(result)
            break

# Print the number of valid configurations
print(f"DONE: {len(last_configurations)}")

[[[2 1 1 2 2]
  [2 1 1 1 1]
  [2 1 1 1 1]
  [2 2 2 2 2]
  [0 2 2 2 2]]

 [[2 1 1 2 2]
  [2 1 1 1 1]
  [2 1 1 1 1]
  [2 0 1 1 1]
  [2 2 1 1 1]]

 [[1 1 1 2 2]
  [1 1 1 1 1]
  [1 1 0 1 1]
  [1 1 1 1 1]
  [2 2 1 1 1]]

 [[1 1 1 2 2]
  [1 1 1 0 2]
  [1 1 1 1 2]
  [1 1 1 1 2]
  [2 2 1 1 2]]

 [[2 2 2 2 0]
  [2 2 2 2 2]
  [1 1 1 1 2]
  [1 1 1 1 2]
  [2 2 1 1 2]]]
DONE: 1
